In [1]:
import json
import sys
import ipytest
from pathlib import Path
import logging
import warnings

%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

logging.getLogger('sagemaker').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [3]:
import sagemaker
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/kmitura/.config/sagemaker/config.yaml


In [4]:
import os
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

DUMMY_ROLE = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-11111111111111"
BUCKET = os.environ["BUCKET"]
S3_LOCATION = f"s3://{BUCKET}/football"
LOCAL_MODE = False

role = os.environ["ROLE"]

architecture = !(uname -m)
IS_ARM64_ARCHITECTURE = architecture[0] == "arm64"

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=BUCKET),
        "instance_type": "local",
        "image": "sagemaker-xgboost-training-toolkit-local" if IS_ARM64_ARCHITECTURE else None
    }
else:
    config = {
        "session": PipelineSession(default_bucket=BUCKET) if not LOCAL_MODE else None,
        "instance_type": "ml.c5.xlarge",
        "image": None,
    }

config["framework_version"] = "1.7-1"
config["py_version"] = "py310"

In [5]:
from sagemaker.s3 import S3Uploader

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

# S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])
# S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])

In [6]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [7]:
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.sklearn import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.retry import (
    SageMakerJobExceptionTypeEnum,
)

retry_policy = {
    "ExceptionType": SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
    "IntervalSeconds": 2,
    "BackoffRate": 2,
    "MaxAttempts": 5,
    "ExpireAfterMin": 1
}

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

processor = SKLearnProcessor(
    base_job_name="split-and-transform-data",
    framework_version="1.2-1",
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config['session'],
)

split_and_transform_data_step = ProcessingStep(
    name="split-and-transform-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",
        inputs=[
            ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),
        ],
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),
            ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline"),
            ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline"),
        ]
    ),
    cache_config=cache_config,
)

In [8]:
from sagemaker.workflow.pipeline import Pipeline

session1_pipeline = Pipeline(
    name="session1-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config['session'],
)

session1_pipeline.upsert(role)
# session1_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session1-pipeline',
 'ResponseMetadata': {'RequestId': '1302ed00-2029-47d4-aa80-2c40bfc2918b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1302ed00-2029-47d4-aa80-2c40bfc2918b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Sat, 27 Apr 2024 08:36:38 GMT'},
  'RetryAttempts': 0}}

In [9]:
from sagemaker.xgboost import XGBoost

use_spot_instances = True and not LOCAL_MODE
max_run = 1000
max_wait = 1200 if use_spot_instances else None
instance_type = "ml.m5.2xlarge"

xgboost = XGBoost(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    role=role,
    instance_count=1,
    instance_type=config['instance_type'],
    framework_version=config['framework_version'],
    disable_profiler=True,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    sagemaker_session=config['session']
)

In [10]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput


def create_training_step(estimator):
    """Create a SageMaker TrainingStep using the provided estimator."""
    return TrainingStep(
        name="train-model",
        step_args=estimator.fit(
            inputs={
                "train": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "train"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "validation"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
            },
        ),
        cache_config=cache_config,
    )

train_model_step = create_training_step(xgboost)

In [11]:
USE_TUNING_STEP = False and not LOCAL_MODE

In [12]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'eta': ContinuousParameter(min_value=0.05, max_value=0.3, scaling_type="Logarithmic"),
    'max_depth': IntegerParameter(min_value=5, max_value=15, scaling_type="Auto"),
    'subsample': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Auto"),
    'colsample_bytree': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(min_value=5, max_value=12, scaling_type="Logarithmic"),
    'alpha': ContinuousParameter(min_value=1, max_value=10, scaling_type="Logarithmic"),
    'min_child_weight': ContinuousParameter(min_value=0.4, max_value=1.0, scaling_type="Auto"),
}

objective_type = "Maximize"
metric_definitions = [
    {'Name': 'validation:logloss',
     'Regex': r".*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*"
     },
    {'Name': 'validation:f1',
     'Regex': 'F1 score: ([0-9\\.]+)'
     },
]
metric_name = "validation:f1"
strategy = "Bayesian"

tuner = HyperparameterTuner(
    estimator=xgboost,
    objective_metric_name=metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    early_stopping_type='Auto',
)

In [13]:
from sagemaker.workflow.steps import TuningStep

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        },
    ),
    cache_config=cache_config
)

In [14]:
session2_pipeline = Pipeline(
    name="session2-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session2_pipeline.upsert(role_arn=role)
# session2_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session2-pipeline',
 'ResponseMetadata': {'RequestId': 'f235e272-122d-4e73-8bbb-2ce3f72a3c41',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f235e272-122d-4e73-8bbb-2ce3f72a3c41',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Sat, 27 Apr 2024 08:36:41 GMT'},
  'RetryAttempts': 0}}

In [15]:
from sagemaker.xgboost import XGBoostProcessor

evaluation_processor = XGBoostProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

In [16]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [17]:
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

if USE_TUNING_STEP:
    model_assets = tune_model_step.get_top_model_s3_uri(
        top_k=0,
        s3_bucket=config["session"].default_bucket(),
    )

In [18]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            ProcessingInput(
                source=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

In [19]:
session3_pipeline = Pipeline(
    name="session3-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session3_pipeline.upsert(role_arn=role)
# session3_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session3-pipeline',
 'ResponseMetadata': {'RequestId': 'ec8986c1-2e30-4a1e-9b38-c44e0a142760',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ec8986c1-2e30-4a1e-9b38-c44e0a142760',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Sat, 27 Apr 2024 08:36:47 GMT'},
  'RetryAttempts': 0}}

In [20]:
BASIC_MODEL_PACKAGE_GROUP = "basic-football"

In [21]:
from sagemaker.xgboost.model import XGBoostModel

xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

In [22]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.functions import Join

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs[
                    "evaluation"
                ].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    ),
)

In [24]:
from sagemaker.workflow.model_step import ModelStep

def create_registration_step(
        model,
        model_package_group_name,
        approval_status="Approved",
        content_types=["text/csv"],
        response_types=["text/csv"],
        model_metrics=None,
        drift_check_baselines=None,
):
    """Create a Registration Step using the supplied parameters."""
    return ModelStep(
        name="register",
        step_args=model.register(
            model_package_group_name=model_package_group_name,
            approval_status=approval_status,
            model_metrics=model_metrics,
            drift_check_baselines=drift_check_baselines,
            content_types=content_types,
            response_types=response_types,
            inference_instances=[config["instance_type"]],
            transform_instances=[config["instance_type"]],
            framework_version=config["framework_version"],
            domain="MACHINE_LEARNING",
            task="CLASSIFICATION",
            framework="XGBOOST",
        ),
    )


register_model_step = create_registration_step(
    xgb_model,
    BASIC_MODEL_PACKAGE_GROUP,
    model_metrics=model_metrics,
)

In [24]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo

f1_threshold = ParameterFloat(name="f1_threshold", default_value=0.64)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's f1 result was lower than",
            f1_threshold,
        ],
    ),
)

condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.f1.value",
    ),
    right=f1_threshold,
)

condition_step = ConditionStep(
    name="check-model-f1-result",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [25]:
session5_pipeline = Pipeline(
    name="session5-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session5_pipeline.upsert(role_arn=role)
# session5_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session5-pipeline',
 'ResponseMetadata': {'RequestId': 'c2265b72-8d6a-4f98-a0b4-fcdd76836742',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2265b72-8d6a-4f98-a0b4-fcdd76836742',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Thu, 25 Apr 2024 18:32:11 GMT'},
  'RetryAttempts': 0}}

In [26]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=BASIC_MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = (
    response["ModelPackageSummaryList"][0]
    if response["ModelPackageSummaryList"]
    else None
)

# package

In [27]:
lambda_role_name = "lambda-deployment-role"
lambda_role_arn = None

try:
    response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(
            {
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": ["lambda.amazonaws.com", "events.amazonaws.com"],
                        },
                        "Action": "sts:AssumeRole",
                    },
                ],
            },
        ),
        Description="Lambda Endpoint Deployment",
    )

    lambda_role_arn = response["Role"]["Arn"]

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole",
        RoleName=lambda_role_name,
    )

    iam_client.attach_role_policy(
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess",
        RoleName=lambda_role_name,
    )

    print(f'Role "{lambda_role_name}" created with ARN "{lambda_role_arn}".')
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = response["Role"]["Arn"]
    print(f'Role "{lambda_role_name}" already exists with ARN "{lambda_role_arn}".')

Role "lambda-deployment-role" already exists with ARN "arn:aws:iam::284415450706:role/lambda-deployment-role".


In [28]:
from sagemaker.lambda_helper import Lambda

ENDPOINT = "football-endpoint"
DATA_CAPTURE_DESTINATION = f"{S3_LOCATION}/monitoring/data-capture"
DATA_CAPTURE_PERCENTAGE = 100

deploy_lambda_fn = Lambda(
    function_name="deployment_fn",
    execution_role_arn=lambda_role_arn,
    script=(CODE_FOLDER / "lambda" / "lambda.py").as_posix(),
    handler="lambda.lambda_handler",
    timeout=600,
    session=sagemaker_session,
    runtime="python3.11",
    environment={
        "Variables": {
            "ENDPOINT": ENDPOINT,
            "DATA_CAPTURE_DESTINATION": DATA_CAPTURE_DESTINATION,
            "DATA_CAPTURE_PERCENTAGE": str(DATA_CAPTURE_PERCENTAGE),
            "ROLE": role,
        },
    },
)

deploy_lambda_fn_response = deploy_lambda_fn.upsert()
deploy_lambda_fn_response

{'ResponseMetadata': {'RequestId': '1e31df91-5009-46f8-8cad-9d21b72db5ca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 25 Apr 2024 18:32:14 GMT',
   'content-type': 'application/json',
   'content-length': '1606',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1e31df91-5009-46f8-8cad-9d21b72db5ca'},
  'RetryAttempts': 0},
 'FunctionName': 'deployment_fn',
 'FunctionArn': 'arn:aws:lambda:eu-north-1:284415450706:function:deployment_fn',
 'Runtime': 'python3.11',
 'Role': 'arn:aws:iam::284415450706:role/lambda-deployment-role',
 'Handler': 'lambda.lambda_handler',
 'CodeSize': 3588,
 'Description': '',
 'Timeout': 600,
 'MemorySize': 128,
 'LastModified': '2024-04-25T18:32:14.000+0000',
 'CodeSha256': '8fXy8oyIt19aH/yecnKKVziTbMCHFI+ILbXFnyhhXvw=',
 'Version': '$LATEST',
 'Environment': {'Variables': {'ROLE': 'arn:aws:iam::284415450706:role/service-role/AmazonSageMaker-ExecutionRole-20240309T101533',
   'DATA_CAPTURE_PERCENTAGE': '100',
   'DATA_CAPTURE_DESTINATION': 

In [29]:
from sagemaker.workflow.lambda_step import LambdaStep


def create_deployment_step(register_model_step):
    """Create a Deploy Step using the supplied parameters."""
    return LambdaStep(
        name="deploy",
        lambda_func=deploy_lambda_fn,
        inputs={
            "model_package_arn": register_model_step.properties.ModelPackageArn,
        },
    )


deploy_step = create_deployment_step(register_model_step)

In [30]:
condition_step = ConditionStep(
    name="check-model-f1-result",
    conditions=[condition],
    if_steps=[register_model_step, deploy_step],
    else_steps=[fail_step],
)

In [31]:
session6_pipeline = Pipeline(
    name="session6-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session6_pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session6-pipeline',
 'ResponseMetadata': {'RequestId': 'd5ac3b0a-c4ca-4273-be38-10668a04beeb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd5ac3b0a-c4ca-4273-be38-10668a04beeb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Thu, 25 Apr 2024 18:32:17 GMT'},
  'RetryAttempts': 0}}

In [32]:
# session6_pipeline.start()

In [34]:
PENDING_MODEL_PACKAGE_GROUP = "pending-football"

In [35]:
event_pattern = f"""
{{
  "source": ["aws.sagemaker"],
  "detail-type": ["SageMaker Model Package State Change"],
  "detail": {{
    "ModelPackageGroupName": ["{PENDING_MODEL_PACKAGE_GROUP}"],
    "ModelApprovalStatus": ["Approved"]
  }}
}}
"""

In [36]:
rule_name = "PendingModelApprovedRule"

events_client = boto3.client("events")
rule_response = events_client.put_rule(
    Name=rule_name,
    EventPattern=event_pattern,
    State="ENABLED",
    RoleArn=role,
)

response = events_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            "Id": "1",
            "Arn": deploy_lambda_fn_response["FunctionArn"],
        },
    ],
)

lambda_function_name = deploy_lambda_fn_response["FunctionName"]
lambda_client = boto3.client("lambda")

try:
    response = lambda_client.add_permission(
        Action="lambda:InvokeFunction",
        FunctionName=lambda_function_name,
        Principal="events.amazonaws.com",
        SourceArn=rule_response["RuleArn"],
        StatementId="EventBridge",
    )
except lambda_client.exceptions.ResourceConflictException:
    print(f'Function "{lambda_function_name}" already has the specified permission.')

Function "deployment_fn" already has the specified permission.


In [37]:
register_model_step = create_registration_step(
    xgb_model,
    PENDING_MODEL_PACKAGE_GROUP,
    approval_status="PendingManualApproval",
    model_metrics=model_metrics,
)

condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [38]:
session7_pipeline = Pipeline(
    name="session7-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session7_pipeline.upsert(role_arn=role)
# session7_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session7-pipeline',
 'ResponseMetadata': {'RequestId': '89600be4-c71f-43ac-be5f-4b2cf74eb073',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '89600be4-c71f-43ac-be5f-4b2cf74eb073',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Thu, 25 Apr 2024 18:32:21 GMT'},
  'RetryAttempts': 0}}

In [39]:
transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [40]:
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=role,
)
xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version='1.2-1',
    sagemaker_session=config["session"],
    role=role,
)

from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=role,
)

In [41]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline-football"

In [42]:
register_model_step = create_registration_step(
    pipeline_model,
    PIPELINE_MODEL_PACKAGE_GROUP,
    content_types=["text/csv", ],
    response_types=["text/csv", ],
    model_metrics=model_metrics,
)

deploy_step = create_deployment_step(register_model_step)

condition_step = ConditionStep(
    name="check-model-f1-score",
    conditions=[condition],
    if_steps=[register_model_step, deploy_step],
    else_steps=[fail_step],
)

In [43]:
session_pipeline = Pipeline(
    name="session-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session_pipeline.upsert(role_arn=role)
# session_pipeline.start()

{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session-pipeline',
 'ResponseMetadata': {'RequestId': 'e47d7b33-5eba-4532-9df7-ceb1aa3b7ba6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e47d7b33-5eba-4532-9df7-ceb1aa3b7ba6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '85',
   'date': 'Thu, 25 Apr 2024 18:32:26 GMT'},
  'RetryAttempts': 0}}

In [45]:
GROUND_TRUTH_LOCATION = f"{S3_LOCATION}/monitoring/groundtruth"
DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"
MODEL_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/model-quality"

In [46]:
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig, QualityCheckStep
from sagemaker.model_monitor.dataset_format import DatasetFormat

data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=config['session'],
        role=config['role'],
    ),

    quality_check_config=DataQualityCheckConfig(
        baseline_dataset=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,
        dataset_format=DatasetFormat.csv(header=True, output_columns_position="END"),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),
    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

KeyError: 'role'

In [ ]:
from sagemaker.workflow.functions import Join

transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [ ]:
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)
xgb_model = XGBoostModel(
    model_data=tune_model_step.get_top_model_s3_uri(
        top_k=0, s3_bucket=config["session"].default_bucket()
    ) if USE_TUNING_STEP else train_model_step.properties.ModelArtifacts.S3ModelArtifacts,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=config["role"],
)

post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)

from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=config['role'],
)

In [ ]:
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.transformer import Transformer

create_model_step = ModelStep(
    name="create-model-step",
    display_name="create-model",
    step_args=pipeline_model.create(
        instance_type="ml.m5.xlarge"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    instance_type=config["instance_type"],
    instance_count=1,
    strategy="MultiRecord",
    accept="text/csv",
    assemble_with="Line",
    output_path=f"{S3_LOCATION}/transform",
    sagemaker_session=config["session"]
)

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    step_args=transformer.transform(
        # We will use the baseline set we generated when we split the data.
        # This set corresponds to the test split before the transformation step.
        data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["baseline"].S3Output.S3Uri,

        join_source="Input",
        split_type="Line",
        content_type="text/csv",

        # We want to output the first and the last field from the joint set.
        # The first field corresponds to the groundtruth, and the last field
        # corresponds to the prediction.
        output_filter="$[0,-1]",
    ),
    cache_config=cache_config
)

In [ ]:
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",

    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=pipeline_session,
        role=execution_role,
    ),

    quality_check_config=ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,
        dataset_format=DatasetFormat.csv(header=False),

        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="BinaryClassification",

        # Since the data doesn't have headers, SageMaker will autocreate headers for it.
        # _c0 corresponds to the first column, and _c1 corresponds to the second column.
        ground_truth_attribute="_c0",
        inference_attribute="_c1",

        output_s3_uri=MODEL_QUALITY_LOCATION,
    ),

    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.drift_check_baselines import DriftCheckBaselines

model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),

    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

In [ ]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline"

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=pipeline_model.register(
        model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="PendingManualApproval",
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="XGBOOST",
        framework_version=config["framework_version"],
    ),
)

In [ ]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep

accuracy_threshold = ParameterFloat(name="f1_score_threshold", default_value=0.64)

condition_step = ConditionStep(
    name="check-model-f1_score",
    conditions=[
        ConditionGreaterThanOrEqualTo(
            left=JsonGet(
                step_name=evaluate_model_step.name,
                property_file=evaluation_report,
                json_path="metrics.f1.value",
            ),
            right=accuracy_threshold,
        )
    ],
    if_steps=[],

    else_steps=[
        FailStep(
            name="fail",
            error_message=Join(
                on=" ",
                values=[
                    "Execution failed because the model's accuracy was lower than",
                    accuracy_threshold,
                ],
            ),
        )
    ],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    condition_step.if_steps.extend(
        [
            create_model_step,
            generate_test_predictions_step,
            model_quality_baseline_step,
            register_model_step,
        ]
    )

In [ ]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline = Pipeline(
    name="session4-pipeline",
    parameters=[
        dataset_location,
        accuracy_threshold,
    ],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True),
    sagemaker_session=config["session"],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    pipeline.steps.extend([data_quality_baseline_step])

pipeline.upsert(role_arn=config["role"])

In [ ]:
pipeline.start()

In [46]:
from sagemaker.base_serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
from sagemaker import Predictor
import pandas as pd

sample = [
    {
        "player_rating_home_player_1": 89,
        "player_rating_home_player_2": 79,
        "player_rating_home_player_3": 59,
        "player_rating_home_player_4": 69,
        "player_rating_home_player_5": 69,
        "player_rating_home_player_6": 79,
        "player_rating_home_player_7": 69,
        "player_rating_home_player_8": 79,
        "player_rating_home_player_9": 69,
        "player_rating_home_player_10": 89,
        "player_rating_home_player_11": 89,
        "player_rating_away_player_1": 79,
        "player_rating_away_player_2": 79,
        "player_rating_away_player_3": 79,
        "player_rating_away_player_4": 79,
        "player_rating_away_player_5": 79,
        "player_rating_away_player_6": 79,
        "player_rating_away_player_7": 80,
        "player_rating_away_player_8": 80,
        "player_rating_away_player_9": 71,
        "player_rating_away_player_10": 83,
        "player_rating_away_player_11": 80,
        "ewm_home_team_goals": 5.54,
        "ewm_away_team_goals": 0.61,
        "ewm_home_team_goals_conceded": 0.26,
        "ewm_away_team_goals_conceded": 4.76,
        "points_home": 30,
        "points_away": 15,
        "home_weighted_wins": 5.377149515625,
        "away_weighted_wins": 2.5561203576634663,
        "avg_home_team_rating": 84.18,
        "avg_away_team_rating": 70.91,
        "home_streak_wins": 11.75,
        "away_streak_wins": 5.58,
        "ewm_shoton_home": 3.55,
        "ewm_shoton_away": 1.805,
        "ewm_possession_home": 53.639,
        "ewm_possession_away": 20.03,
        "avg_home_rating_attack": 71.33,
        "avg_away_rating_attack": 78.83,
        "avg_away_rating_defence": 79.0,
        "avg_home_rating_defence": 71.0,
        "average_rating_home": 89.18181818181819,
        "average_rating_away": 78.9090909090909,
        "num_top_players_home": 0,
        "num_top_players_away": 0,
        "ewm_home_team_goals_conceded_x_ewm_shoton_home": 4.473,
        "attacking_strength_home": 80.233606557377048,
        "attacking_strength_away": 31.40637450199203,
        "attacking_strength_diff": -2.172767944614982
    },
    {
        "player_rating_home_player_1": 89,
        "player_rating_home_player_2": 79,
        "player_rating_home_player_3": 59,
        "player_rating_home_player_4": 69,
        "player_rating_home_player_5": 69,
        "player_rating_home_player_6": 79,
        "player_rating_home_player_7": 69,
        "player_rating_home_player_8": 79,
        "player_rating_home_player_9": 69,
        "player_rating_home_player_10": 89,
        "player_rating_home_player_11": 89,
        "player_rating_away_player_1": 79,
        "player_rating_away_player_2": 79,
        "player_rating_away_player_3": 79,
        "player_rating_away_player_4": 79,
        "player_rating_away_player_5": 79,
        "player_rating_away_player_6": 79,
        "player_rating_away_player_7": 80,
        "player_rating_away_player_8": 80,
        "player_rating_away_player_9": 71,
        "player_rating_away_player_10": 83,
        "player_rating_away_player_11": 80,
        "ewm_home_team_goals": 5.54,
        "ewm_away_team_goals": 0.61,
        "ewm_home_team_goals_conceded": 0.26,
        "ewm_away_team_goals_conceded": 4.76,
        "points_home": 30,
        "points_away": 15,
        "home_weighted_wins": 5.377149515625,
        "away_weighted_wins": 2.5561203576634663,
        "avg_home_team_rating": 84.18,
        "avg_away_team_rating": 70.91,
        "home_streak_wins": 11.75,
        "away_streak_wins": 5.58,
        "ewm_shoton_home": 3.55,
        "ewm_shoton_away": 1.805,
        "ewm_possession_home": 53.639,
        "ewm_possession_away": 20.03,
        "avg_home_rating_attack": 71.33,
        "avg_away_rating_attack": 78.83,
        "avg_away_rating_defence": 79.0,
        "avg_home_rating_defence": 71.0,
        "average_rating_home": 89.18181818181819,
        "average_rating_away": 78.9090909090909,
        "num_top_players_home": 0,
        "num_top_players_away": 0,
        "ewm_home_team_goals_conceded_x_ewm_shoton_home": 4.473,
        "attacking_strength_home": 80.233606557377048,
        "attacking_strength_away": 31.40637450199203,
        "attacking_strength_diff": -2.172767944614982
    }
]

sample_df = pd.DataFrame(sample)

csv_data = sample_df.to_csv(index=False, header=False)

predictor = Predictor(
    endpoint_name=ENDPOINT,
    serializer=CSVSerializer(),  # Ensure the serializer is set for CSV
    deserializer=JSONDeserializer(),
    sagemaker_session=sagemaker_session,
)

try:
    response = predictor.predict(csv_data)
    print(json.dumps(response, indent=2))
except Exception as e:
    print("Error:", e)


[
  {
    "prediction": "home_win",
    "confidence": 0.5134705901145935
  },
  {
    "prediction": "home_win",
    "confidence": 0.5134705901145935
  }
]
